In [1]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(1337)  # for reproducibility

from theano import function

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape, Layer, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D
from keras.utils import np_utils
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l2

from seya.layers.variational import VariationalDense as VAE
from seya.layers.convolutional import GlobalPooling2D
from seya.utils import apply_model

from agnez import grid2d, img_grid

batch_size = 100
nb_epoch = 3
code_size = 200

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 5
nb_classes = 10

# the data, shuffled and split between tran and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_train = X_train.astype("float32") - 128
X_test = X_test.astype("float32") - 128
X_train /= 128
X_test /= 128

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_valid = X_train[50000:]
Y_valid = Y_train[50000:]
X_train = X_train[:50000]
Y_train = Y_train[:50000]

Using Theano backend.
X_train shape: (60000, 1, 28, 28)
60000 train samples
10000 test samples


Using gpu device 0: GeForce GTX 680 (CNMeM is disabled)


In [2]:
def wtall(X):
    M = K.max(X, axis=(2, 3), keepdims=True)
    R = K.switch(K.equal(X, M), X, 0.)
    return R

In [3]:
enc = Sequential()
enc.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        #W_regularizer=l2(.0005),
                        border_mode='same',
                        input_shape=(1, img_rows, img_cols)))
#enc.add(Dropout(.5))
enc.add(Activation('relu'))
enc.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='same'))
enc.add(Activation('relu'))

pool_shape = enc.output_shape

enc.add(Lambda(function=wtall, output_shape=pool_shape))

In [4]:
dec = Sequential()
dec.add(Convolution2D(1, 11, 11, border_mode='same',
                     input_shape=pool_shape[1:]))
# dec.add(Activation('linear'))
dec.add(Flatten())

In [ ]:
model = Sequential()
model.add(enc)
model.add(dec)

model.compile(loss='mse', optimizer='adam')

In [ ]:
#cbk = ModelCheckpoint('vae/vae.hdf5', save_best_only=True)

try:
    model.fit(X_train, X_train.reshape((-1, 784)), batch_size=batch_size, nb_epoch=nb_epoch, verbose=1,
          validation_data=(X_valid, X_valid.reshape((-1, 784)))
              #,callbacks=[cbk]
             )
except:
    pass

Train on 50000 samples, validate on 10000 samples
Epoch 1/3
19700/50000 [==========>...................] - ETA: 28s - loss: 0.3473

# Visualize first layers

In [ ]:
W = np.asarray(K.eval(enc.layers[0].W))
W = W.reshape((32, -1))
I = grid2d(W)

In [ ]:
plt.imshow(I)

# Visualize decoder

In [ ]:
W = np.asarray(K.eval(dec.layers[0].W))
W = W.transpose(1, 0, 2, 3)
I = img_grid(W)
plt.imshow(I)